In [106]:
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch.optim as optim
from torch.autograd import Variable
import torch
from tqdm import tqdm
import sys

In [107]:
sys.path.insert(0,'..')
import bitmap
import itertools
from forward_prediction import forward_model

In [108]:
class ReverseNet(nn.Module):
    def __init__(self):
        super(ReverseNet, self).__init__()
        # in channels, out channels, kernel size
        self.conv1 = nn.Conv2d(1, 16, (5, 5), padding=(2, 2), padding_mode='circular')
        self.activ1 = nn.PReLU()
        self.conv2 = nn.Conv2d(16, 8, (5, 5), padding=(2, 2), padding_mode='circular')
        self.activ2 = nn.PReLU()
        self.conv3 = nn.Conv2d(8, 4, (3, 3), padding=(1, 1), padding_mode='circular')
        self.activ3 = nn.PReLU()
        self.conv4 = nn.Conv2d(4, 1, (3, 3), padding=(1, 1), padding_mode='circular')

    def forward(self, x):
        x = self.reverse(x)
        return x

    def reverse(self, x):
        x = self.activ1(self.conv1(x))
        x = self.activ2(self.conv2(x))
        x = self.activ3(self.conv3(x))
        x = torch.sigmoid(self.conv4(x))
        return x

In [109]:
net = ReverseNet()
print(net)

ReverseNet(
  (conv1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), padding_mode=circular)
  (activ1): PReLU(num_parameters=1)
  (conv2): Conv2d(16, 8, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), padding_mode=circular)
  (activ2): PReLU(num_parameters=1)
  (conv3): Conv2d(8, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=circular)
  (activ3): PReLU(num_parameters=1)
  (conv4): Conv2d(4, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=circular)
)


In [110]:
criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters()) #, lr=0.001, momentum=0.9)

In [133]:
# Generate training data
N = 25600
# train_data_gen = bitmap.generate_train_set(N, 41, max_delta=1)
train_data_gen = bitmap.generate_train_set(N, 41, min_delta=5, max_delta=5)
deltas, start_boards, stop_boards = map(list, zip(*list(train_data_gen)))

In [134]:
X = Variable(torch.tensor(stop_boards).view(N, 1, 25, 25).float(), requires_grad=True)
y = Variable(torch.tensor(start_boards).view(N, 1, 25, 25).float())
num_epochs = 100
batch_size = 128
for epoch in tqdm(range(num_epochs)): 
    permutation = torch.randperm(X.size()[0])
    running_loss = 0.0
    for i in range(0, X.size()[0], batch_size):
        indices = permutation[i:i+batch_size]
        batch = X[indices]
        target = y[indices]
        
        optimizer.zero_grad()
        outputs = net(batch)
        # loss = criterion(outputs, batch)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 1000 == 0 and i > 0:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 10))
        running_loss = 0.0

  0%|          | 0/100 [00:00<?, ?it/s]

[1, 16001] loss: 0.035


  1%|          | 1/100 [00:18<30:47, 18.66s/it]

[2, 16001] loss: 0.034


  2%|▏         | 2/100 [00:35<29:48, 18.26s/it]

...

 98%|█████████▊| 98/100 [31:34<00:37, 18.81s/it]

[99, 16001] loss: 0.036


 99%|█████████▉| 99/100 [31:53<00:18, 18.84s/it]

[100, 16001] loss: 0.037


100%|██████████| 100/100 [32:11<00:00, 19.32s/it]


In [139]:
# Generate testing data
N_test = 12800
# test_data_gen = bitmap.generate_test_set(N_test, 45, max_delta=1)
test_data_gen = bitmap.generate_test_set(N_test, 45, min_delta=5, max_delta=5)
delta, test_stop_boards = map(list, zip(*list(test_data_gen)))

In [141]:
def get_mae(stop_boards, n, delta=1):
    stop_boards_tensor = torch.tensor(stop_boards).view(n, 1, 25, 25).float()
    with torch.no_grad():
        net.eval()
        # Make prediction
        output = (net(stop_boards_tensor) > 0.5).int()
        # Evolve one step forward
        predicted_stop_board = output
        for _ in range(delta):
            predicted_stop_board = (forward_model.forward(predicted_stop_board) > 0.5).int()
        error = torch.sum(predicted_stop_board != stop_boards_tensor.int())
        return error / (n * 25 * 25)

In [142]:
# Training data MAE
train_mae = get_mae(stop_boards, N, 5).item()
print("The training data MAE is {:.6f}.".format(train_mae))

# Test data MAE
test_mae = get_mae(test_stop_boards, N_test, 5).item()
print("The test data MAE is {:.6f}.".format(test_mae))

The training data MAE is 0.139022.
The test data MAE is 0.138522.


In [132]:
predicted_stop_board

NameError: name 'predicted_stop_board' is not defined